In [ ]:
import numpy as np
import pandas as pd
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input, Flatten
from skimage import morphology, color, io, exposure, transform, io, img_as_float, exposure
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import cv2
import os

In [ ]:
viralPneumonia=sorted(list(paths.list_images('/Users/skikkisetti/bacterial_pneumonia_BIG/data/')))
for file in viralPneumonia:
    print(file)




In [ ]:
import numpy as np
import pandas as pd
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input
from skimage import morphology, color, io, exposure



def masked(img, gt, mask, alpha=0):
    """Returns image with GT lung field outlined with red, predicted lung field
    filled with blue."""
    rows, cols = img.shape
    color_mask = np.zeros((rows, cols, 3))
    color_mask[mask == 1] = [1, 1, 1]
    img_color = np.dstack((img, img, img))

    img_hsv = img_color
    color_mask_hsv = color_mask

    img_masked = color_mask_hsv
    return img_masked

def remove_small_regions(img, size):
    """Morphologically removes small (less than size) connected regions of 0s or 1s."""
    img = morphology.remove_small_objects(img, size)
    img = morphology.remove_small_holes(img, size)
    return img


In [ ]:
if __name__ == '__main__':

    # Path to csv-file. File should contain X-ray filenames as first column,
    # mask filenames as second column.
    csv_path = '/Users/skikkisetti/bacterial_pneumonia_BIG/idxbp.csv'
    # Path to the folder with images. Images will be read from path + path_from_csv
    path = '/Users/skikkisetti/bacterial_pneumonia_BIG/lungseg/'

    df = pd.read_csv(csv_path)

    # Load test data
    im_shape = (256, 256)
    X, y = loadDataGeneral(df, path, im_shape)

    n_test = X.shape[0]
    inp_shape = X[0].shape
   


    model = 'trained_model.hdf5'
    UNet = load_model(model)

    # For inference standard keras ImageGenerator is used.
    test_gen = ImageDataGenerator(rescale=1.)


    i = 0
    for xx, yy in test_gen.flow(X, y, batch_size=1):
        img = exposure.rescale_intensity(np.squeeze(xx), out_range=(0,1))
        pred = UNet.predict(xx)[..., 0].reshape(inp_shape[:2])
        mask = yy[..., 0].reshape(inp_shape[:2])

        # Binarize masks
        gt = mask > 0.5
        pr = pred > 0.5

        # Remove regions smaller than 2% of the image
        pr = remove_small_regions(pr, 0.02 * np.prod(im_shape))
        string=df.iloc[i][0]
        ind=string.rfind('/')
        string=string[ind:]
        io.imsave('mask/{}'.format(string), masked(img, gt, pr, 1))
        io.imsave('image/{}'.format(string), img)

        i += 1
        if i == n_test:
            break




In [ ]:
def loadDataGeneral(df, path, im_shape):
    """Function for loading arbitrary data in standard formats"""
    from skimage import transform, io, img_as_float, exposure
    X, y,x = [], [], []
    for i, item in df.iterrows():
        img = img_as_float(io.imread(item[0]))
        mask = io.imread(path + item[1])
        img = transform.resize(img, im_shape)

        img=color.rgb2gray(img)
        img = exposure.equalize_hist(img)
        img = np.expand_dims(img, -1)
        mask=color.rgb2gray(mask)
        mask = transform.resize(mask, im_shape)
        mask = np.expand_dims(mask, -1)
        X.append(img)
        y.append(mask)
    X = np.array(X)
    y = np.array(y)
    X -= X.mean()
    X /= X.std()
    print ('### Dataset loaded')
    print ('\t{}'.format(path))
    print ('\t{}\t{}'.format(X.shape, y.shape))
    print ('\tX:{:.1f}-{:.1f}\ty:{:.1f}-{:.1f}\n'.format(X.min(), X.max(), y.min(), y.max()))
    print ('\tX.mean = {}, X.std = {}'.format(X.mean(), X.std()))
    return X, y
